## Book search

In [ ]:
import pandas as pd

df = pd.read_csv("Goodreads_books_with_genres.csv")
df['publication_date'] = pd.to_datetime(df['publication_date'], format='mixed', errors= 'coerce') ##converts to date format
df['year'] = df['publication_date'].dt.year ##takes only year from the date as a float

def get_user_input():  
    title = input("Book title: ").strip().lower()  
    author = input("Author name: ").strip().lower()  
    genre = input("Genre: ").strip().lower()  
    min_rating = float(input("Minimum rating (0-5): ") or 0)  ##ratings are from 0 to 5
    max_rating = float(input("Maximum rating (0-5): ") or 5)
    max_pages = input("Maximum pages: ") or None
    date_published = input("Year published: ") or None ##year or no entry, float applied in function
    message = "Didn't find what you were looking for? Try suggestions below!"
    
    filtered = df[
        (df['average_rating'] >= min_rating) &  ##searches and limits average_rating to min and max rating entered 
        (df['average_rating'] <= max_rating)]
    try:
        if max_pages is not None:
            filtered = filtered[filtered['num_pages'] <= float(max_pages)]
        else: max_pages = str("")  ##if no number is entered it does not return none but rather empty string
    except ValueError: ##this handles error where user inputs number in non-numerical form, e.g. ten instead of 10
        print("WARNING! Incorrect number format for page number. Please enter numerical value.")
    
    try:    
        if date_published is not None:  
            filtered = filtered[filtered['year'] == float(date_published)]
        else: date_published = str("") 
    except ValueError: ##handles error with year format not being a number
        print("WARNING! Incorrect number format for year. Please enter numerical value.")
    
    if title:
        filtered = filtered[filtered['Title'].str.lower().str.contains(title)] ##this finds and filters column title with entered title of book in lowercase
        
    if author:  
        filtered = filtered[filtered['Author'].str.lower().str.contains(author)] 
        
    if genre:  
        filtered = filtered[filtered['genres'].str.lower().str.contains(r'\b' + genre + r'\b', regex=True, na=False)]
    
    if filtered.empty:  ##this reports back when the table is empty, which can be for several reasons
        print("No books found with your search criteria.\n"    
        "Try checking your spelling or adjust your filters.\n")
        
    print(f"Showing results for: \n\nBook title: {title} \nAuthor name: {author} \nGenre: {genre}" 
          f"\nBook rating range (min-max): {min_rating} - {max_rating} \nMaximum page count: {max_pages}" 
          f"\nYear published: {date_published} \n\n{message}")
    return filtered[['Title', 'Author', 'genres', 'average_rating', 'num_pages', 'publication_date']] ##prints the table

In [65]:
test_input = get_user_input()
test_input

No books found with your search criteria.
Try checking your spelling or adjust your filters.

Showing results for: 

Book title: baicoa 
Author name:  
Genre: 
Book rating range (min-max): 0.0 - 5.0 
Maximum page count: 
Year published:  

Didn't find what you were looking for? Try suggestions below!


,Title,Author,genres,average_rating,num_pages,publication_date


## Book suggestions

#### Top 10 best rated books by genre

In [68]:
def input_suggestions():
    genre = input("Genre: ").strip().lower() 
    filtered = df.sort_values(by= 'average_rating', ascending = False) ##sorts based on the book rating
    if genre:  
        filtered = filtered[filtered['genres'].str.lower().str.contains(r'\b' + genre + r'\b', regex=True, na=False)]
    print(f"Showing results for: {genre}") 
    if filtered.empty:  ##this reports back when the table is empty - genre is mispelled or does not exist in the list
        print("No books found in your genre. Check your spelling or try different genre.\n")
    return filtered[['Title', 'Author', 'genres', 'average_rating','num_pages', 'publication_date']]

In [69]:
test_suggestion = input_suggestions()
test_suggestion.head(10) ##gives 10 top rows from the table

Showing results for: fantasy


,Title,Author,genres,average_rating,num_pages,publication_date
4,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve...",4.78,2690,2004-09-13
6,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil...",4.73,3342,2005-09-12
6297,The Absolute Sandman Volume One,Neil Gaiman/Mike Dringenberg/Chris Bachalo/Mic...,"Sequential Art,Comics;Sequential Art,Graphic N...",4.65,612,2006-11-01
3071,Herbert the Timid Dragon,Mercer Mayer,"Childrens,Picture Books;Childrens;Fantasy,Drag...",4.62,48,1991-07-01
6291,Nausicaä of the Valley of the Wind Vol. 6 (Na...,Hayao Miyazaki/Matt Thorn/Kaori Inoue/Joe Yama...,"Sequential Art,Manga;Sequential Art,Graphic No...",4.62,159,2004-08-10
6292,Nausicaä of the Valley of the Wind Vol. 5 (Na...,Hayao Miyazaki/Matt Thorn/Kaori Inoue/Joe Yama...,"Sequential Art,Manga;Sequential Art,Graphic No...",4.61,151,2004-06-30
6293,Nausicaä of the Valley of the Wind Vol. 7 (Na...,Hayao Miyazaki/Matt Thorn/Kaori Inoue/Joe Yama...,"Sequential Art,Manga;Sequential Art,Graphic No...",4.61,223,2004-09-07
8320,鋼之鍊金術師 10,Hiromu Arakawa/荒川弘/方郁仁,"Sequential Art,Manga;Fantasy;Sequential Art,Co...",4.60,188,2005-03-23
6982,Fullmetal Alchemist Vol. 12 (Fullmetal Alchem...,Hiromu Arakawa/Akira Watanabe,"Fantasy;Sequential Art,Graphic Novels;Sequenti...",4.60,192,2007-03-20
2528,Fullmetal Alchemist Vol. 10,Hiromu Arakawa/Akira Watanabe,"Sequential Art,Manga;Fantasy;Sequential Art,Co...",4.60,200,2006-11-21


#### Try your luck! (Random book)

In [70]:
def random_book():
    random_sugg = pd.DataFrame.sample(df, n=1) ## .sample take 1 random row from the whole dataframe
    return random_sugg [['Title', 'Author', 'genres', 'average_rating','num_pages', 'publication_date']]

In [72]:
test_random = random_book()
test_random

,Title,Author,genres,average_rating,num_pages,publication_date
4196,Conversations With God: An Uncommon Dialogue ...,Neale Donald Walsch,Spirituality;Nonfiction;Religion;Philosophy;Se...,4.21,392,2005-12-03


#### Try your luck, get 10 random books, but choose your preferred genre

In [ ]:
def random_books_genre():
    genre = input("Genre: ").strip().lower() 
    filtered = df
    try:
        if genre:  
            filtered = filtered[filtered['genres'].str.lower().str.contains(r'\b' + genre + r'\b', regex=True, na=False)] 
            filtered_genres = pd.DataFrame.sample(filtered, n=10) ## .sample takes a random 10 rows from the filtered books by genre
            print(f"Showing results for: {genre}")     
        return filtered_genres [['Title', 'Author', 'genres', 'average_rating','num_pages', 'publication_date']]
    except ValueError: ##used try-except here because if mispelled genre is entered it returns error because .sample expects 10 of something
          print("No books found in your genre. Please check spelling of the genre entered.")

In [82]:
test_almost_random = random_books_genre()
test_almost_random

Showing results for: childrens


,Title,Author,genres,average_rating,num_pages,publication_date
6478,Lucie Babbidge's House,Sylvia Cassedy,Fiction;Young Adult;Fantasy;Childrens;Novels,3.73,242,1993-04-01
6549,Racso and the Rats of NIMH (Rats of NIMH #2),Jane Leslie Conly/Leonard B. Lubin,Fantasy;Fiction;Childrens;Animals;Young Adult;...,3.77,288,1988-09-01
6465,The Best Is Yet to Come (The Amazing Days of A...,Anne Mazer,"Childrens;Fiction;Childrens,Middle Grade;Child...",4.05,144,2004-06-01
4580,Anne's House Of Dreams,L.M. Montgomery,"Classics;Fiction;Young Adult;Historical,Histor...",4.14,192,2006-11-03
2115,Farm Animals (A Chunky Book),Phoebe Dunn,"Childrens,Picture Books",3.86,28,1984-02-12
819,Salt in His Shoes: Michael Jordan in Pursuit o...,Deloris Jordan/Kadir Nelson/Roslyn M. Jordan,"Childrens,Picture Books;Biography;Sports,Sport...",4.20,32,2003-11-01
717,Below the Root,Zilpha Keatley Snyder/Alton Raible,Fantasy;Fiction;Young Adult;Science Fiction;Ch...,4.08,231,1975-01-01
147,On Beyond Zebra!,Dr. Seuss,"Childrens,Picture Books;Childrens;Fiction;Poet...",4.04,64,1955-09-12
4871,Homemade Love,bell hooks/Shane W. Evans,"Childrens,Picture Books;Family;Childrens;Poetr...",4.18,32,2002-12-23
2923,Charlotte's Web,E.B. White/Garth Williams,Classics;Fiction;Childrens;Fantasy;Young Adult...,4.17,179,2006-11-01
